# Requirements
pandas >= 0.20

In [5]:
import os
import pandas as pd 
import numpy as np
import re
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns = None

## Let us write some functions to create a ML pipeline which will do below tasks -
### 1. Read CSV raw file and create a Pandas DataFrame
### 2. Feature Extraction - Handling missing values, retrieving best features from the data, normalizing features and addressing class imbalance of labels. This will enable us to create a ML ready dataset 
### 3. Training a Classifier - Grid search, K-fold cross-validation, Naive Bayes algorithm to predict 5 year future plan

In [6]:
# Read CSV raw file
def read_data(csv_filename):
    print("Inside read data\n")
    # set working directory, to easily read the CSV data 
    os.chdir(r"C:\Users\prana\Desktop\stack_overflow_data")
    print("Working directory set to :: " + str(os.getcwd()))
    input = pd.read_csv(csv_filename, error_bad_lines = False)
    return input

In [8]:
df = read_data("survey_results_public.csv")
# Below line might give low_memory Warning because Pandas tries to
# infer the datatypes of columns by reading all rows.
df.head(10)

Inside read data

Working directory set to :: C:\Users\prana\Desktop\stack_overflow_data


c:\users\prana\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,Salary,SalaryType,ConvertedSalary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,IDE,OperatingSystem,NumberMonitors,Methodology,VersionControl,CheckInCode,AdBlocker,AdBlockerDisable,AdBlockerReasons,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,3-5 years,3-5 years,Extremely satisfied,Extremely satisfied,Working as a founder or co-founder of my own c...,"I’m not actively looking, but I am open to new...",Less than a year ago,10.0,7.0,8.0,1.0,2.0,5.0,3.0,4.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,4.0,2.0,5.0,5.0,6.0,7.0,2.0,1.0,4.0,3.0,My job status or other personal status changed,NaN,NaN,Monthly,NaN,KES,Slack,One to three months,"Taught yourself a new language, framework, or ...",The official documentation and/or standards fo...,NaN,To build my professional network,Strongly agree,Strongly agree,Neither Agree nor Disagree,JavaScript;Python;HTML;CSS,JavaScript;Python;HTML;CSS,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,AWS;Azure;Linux;Firebase,AWS;Azure;Linux;Firebase,Django;React,Django;React,Komodo;Vim;Visual Studio Code,Linux-based,1,Agile;Scrum,Git,Multiple times per day,Yes,No,NaN,Strongly agree,Strongly agree,Strongly agree,Saw an online advertisement and then researche...,1.0,5.0,4.0,7.0,2.0,6.0,3.0,Artificial intelligence surpassing human intel...,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, and publicly",Upper management at the company/organization,Yes,10 (Very Likely),Multiple times per day,Yes,I have never participated in Q&A on Stack Over...,"No, I knew that Stack Overflow had a jobs boar...",Yes,NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Between 5:00 - 6:00 AM,9 - 12 hours,1 - 2 hours,Never,Standing desk,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy
1,3,Yes,Yes,Unite

In [9]:
# To check count of records in pandas use df.shape
# df.count counts rows for all non-NA rows
df.shape

(98855, 129)

In [10]:
# check dataframe stats and info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Columns: 129 entries, Respondent to SurveyEasy
dtypes: float64(41), int64(1), object(87)
memory usage: 64.5+ MB


In [51]:
null_df = pd.DataFrame(df.isna().sum()).reset_index()
null_df = null_df.rename(columns={'index':'column_name' , 0 : 'missing_values'})
null_df['percentage_missing'] = 100* null_df.missing_values / df.shape[0]
null_df = null_df.sort_values('percentage_missing', ascending=False)
print(type(null_df))
print(null_df.head())

# null_columns=df.columns[df.isna().any()]
# y = pd.DataFrame(df[null_columns].isna().sum()).reset_index()
# y = y.rename(columns={'index':'column_name' , 0 : 'missing_values'})
# y.head()
#.rename(columns={'days_difference':'day','amount':'total_amount','user_id':'unique_users'})


<class 'pandas.core.frame.DataFrame'>
           column_name  missing_values  percentage_missing
60   TimeAfterBootcamp           92203           93.270952
126         MilitaryUS           83074           84.036215
61    HackathonReasons           73164           74.011431
118   ErgonomicDevices           64797           65.547519
81    AdBlockerReasons           61110           61.817814


,column_name,missing_values
0,Country,412
1,Student,3954
2,Employment,3534
3,FormalEducation,4152
4,UndergradMajor,19819


In [32]:
l = df.dtypes.to_dict()
print(type(l))
for k,v in l.items():
    print(str(k) + "::" + str(v))

<class 'dict'>
LastNewJob::object
SexualOrientation::object
AIInteresting::object
PlatformDesireNextYear::object
CareerSatisfaction::object
LanguageWorkedWith::object
LanguageDesireNextYear::object
DatabaseWorkedWith::object
HoursComputer::object
Student::object
FormalEducation::object
AdsPriorities4::float64
Hobby::object
AssessBenefits8::float64
JobEmailPriorities5::float64
AssessBenefits6::float64
StackOverflowDevStory::object
EthicsChoice::object
SurveyEasy::object
JobContactPriorities2::float64
AssessJob4::float64
OperatingSystem::object
JobContactPriorities3::float64
EducationParents::object
SurveyTooLong::object
AdsPriorities3::float64
Age::object
Exercise::object
AssessBenefits5::float64
StackOverflowJobs::object
Employment::object
OpenSource::object
AdBlockerDisable::object
HypotheticalTools3::object
HypotheticalTools1::object
StackOverflowParticipate::object
AssessBenefits4::float64
AssessJob9::float64
HypotheticalTools2::object
Methodology::object
AdsPriorities5::float64
AdB

In [18]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [19]:
df.infer_objects().dtypes

AttributeError: 'DataFrame' object has no attribute 'infer_objects'

In [20]:
pd.__version__

'0.19.2'

In [1]:
import pandas as pd
pd.__version__

'0.23.4'